In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

# example of PromptTemplate

'''
t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country = "France")

'''
# another way of using PromptTemplate
''' 
t = PromptTemplate(
    template = "what is the capital of {country}",
    input_varibale = ["country"],
)
t.format(country = "France")
'''


# FewShotPromptTemplate

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_prompt = PromptTemplate.from_template("Human: {question} \nAI: {answer}") # "question" and "answer" should be same with the examples!

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt, 
    examples = examples,
    suffix = "Human: What do you know about {country}?", # Our question
    input_variables = ["country"]
)

prompt.format(country = "Germany")
chain = prompt | chat

chain.invoke({
    "country" : "Germany"
})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
{
"country": "France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"country": "Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"country": "Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]



example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?")
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt, 
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country" : "Germany"
})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

# example of PromptTemplate

'''
t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country = "France")

'''
# another way of using PromptTemplate
''' 
t = PromptTemplate(
    template = "what is the capital of {country}",
    input_varibale = ["country"],
)
t.format(country = "France")
'''


# FewShotPromptTemplate

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

# Example of RandomExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice 
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question} \nAI: {answer}") # "question" and "answer" should be same with the examples!

# Example of LengthBasedExampleSelector
 
"""
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80 # how many examples I am going to use
)
"""
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt, 
    example_selector=example_selector,
    suffix = "Human: What do you know about {country}?", # Our question
    input_variables = ["country"]
)

prompt.format(country = "Germany")
chain = prompt | chat

chain.invoke({
    "country" : "Germany"
})